# Text Processing

# Topics
- Overview
- Parsing, Stemming, Lemmatization
- Named Entity Recognition
- Stop Words
- Frequency Analysis
- Workshop: Document Summarization

# References

NLTK book: http://www.nltk.org/book/

spaCy 101: https://spacy.io/usage/spacy-101

# Setup

Run this command from an Anaconda prompt (within the mldds03 environment):

```
(mldds03) conda install nltk spacy scikit-learn pandas
```

### NLTK: NLP toolkit
https://www.nltk.org/install.html

### spaCy: another NLP toolkit

Simpler to use than NLTK (but usually fewer knobs)

https://spacy.io/api/

https://spacy.io/usage/spacy-101

# What is Text Processing?

- A sub-field of Natural Language Processing (NLP)
- Natural Language Processing is ...
 - Teaching machines to understand and produce language (text, speech)
 - A combination of computer science and computational linguistics

# Text Processing Tasks

- Word categorization and tagging: part of speech, type of entity
- Semantic Analysis: finding meanings of documents
- Topic Modeling: finding topics from documents
- Document similarity: comparing if two documents are semantically similar
- etc.

Note: Speech is text processing + acoustic model

# Parsing, Stemming & Lemmatization

- Tokenization: splitting text into words
- Sentence boundary detection: splitting text into sentences
- Stemming: finding word stems
   - stating => state, reference => refer
- Lemmatization: finding the base form of words
   - was => be

## Tokenization

- Segmenting text into words, punctuation, etc.
- Rule-based

### Tokenization with spaCy

![tokenization in spaCy](assets/text/tokenization.svg)

(image: https://spacy.io/usage/spacy-101#annotations-token)

In [ ]:
# Download the English model
# You can find other models here: https://spacy.io/models/en
!python -m spacy download en_core_web_sm

In [ ]:
text = u"This is a test. A quick brown fox jumps over the lazy dog."

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(text)

# sentence tokenizer
for sent in doc.sents:
    print()
    print(sent)

In [ ]:
# word tokenizer
for token in doc:
    print(token.text)

In [ ]:
spacy.explain('DET')

https://spacy.io/api/token

https://spacy.io/api/token#attributes

In [ ]:
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

displacy.render(doc, style='dep', jupyter=True, options={'distance': 140})

### Tokenization with NLTK

http://www.nltk.org/api/nltk.tokenize.html

nltk.tokenize
 - sent_tokenize
 - word_tokenize
 - wordpunc_tokenize


In [ ]:
# Download the Punkt sentence tokenizer
# https://www.nltk.org/_modules/nltk/tokenize/punkt.html

# List of available corpora: http://www.nltk.org/book/ch02.html#tab-corpora
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize

# list of sentences
sent_tokenize(text)

In [ ]:
from nltk.tokenize import word_tokenize

# flat list of words and punctuations
word_tokenize(text)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

sentences = sent_tokenize(text)

# list of lists
[word_tokenize(sentence) for sentence in sentences]

In [ ]:
from nltk.tokenize import wordpunct_tokenize

text2 = "'The time is now 5.30am,' he said."

print(word_tokenize(text2))

print(wordpunct_tokenize(text2))

In [ ]:
# Part of speech tagging
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import sent_tokenize, word_tokenize

sentences = sent_tokenize(text)
sentences = [word_tokenize(sentence) for sentence in sentences]

[nltk.pos_tag(word) for word in sentences]

In [ ]:
spacy.explain('JJ')

#### Twitter-aware tokenizer

`nltk.tokenize.TweetTokenizer`

http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.casual

In [ ]:
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer()
tweet = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"

tknzr.tokenize(tweet)

In [ ]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
tweet = '@remy: This is waaaaayyyy too much for you!!!!!!'

tknzr.tokenize(tweet)

## Stemming vs. Lemmatization

- Stemming uses rule-based heuristics
  - ponies => poni
  - Quicker, but less precision
- Lemmatization uses vocabulary and morphological analysis
  - ponies => pony
  - For English, not much improvement over stemming because context of word use is more important

https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

## Porter Stemmer

- 5 sequential phases of word reductions
- Applies rules such as "sses -> ss", "ies => i"

![stemmers](assets/text/stemmers.png)

(image: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)

### Stemming & Lemmatization with spaCy

`spacy.lemmatizer.Lemmatizer`

https://spacy.io/api/lemmatizer

In [ ]:
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

nlp = spacy.load('en_core_web_sm')
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

doc = nlp(text)

for token in doc:
    print(lemmatizer(token.text, token.pos_))

### Stemming & Lemmatization with NLTK

`nltk.stem`
- `PorterStemmer`
- `WordNetLemmatizer`

http://www.nltk.org/api/nltk.stem.html

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

tokens = word_tokenize(text)

for token in tokens:
    print(stemmer.stem(token))

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

tokens = word_tokenize(text)

for token in tokens:
    print(lemmatizer.lemmatize(token))

## Named Entity Recognition

- Find and classify entities within text
  - Persons
  - Organizations
  - Locations
  - Time expressions
  - Quantities
  - Phone numbers
  - etc
  
- Grammar-based models, trained classifiers

- Can be corpus-dependent, see https://spacy.io/api/annotation#named-entities

### Named Entity Recognition with spaCy

https://spacy.io/api/annotation#named-entities

In [ ]:
nlp = spacy.load('en_core_web_sm')

text3 = u"Flight 224 is scheduled to arrive in Frankfurt at 4pm July 5th, 2018."
doc = nlp(text3)

for entity in doc.ents:
    print(entity.text, entity.label_, entity.start_char, entity.end_char)

In [ ]:
spacy.explain('NORP')

In [ ]:
from spacy import displacy

displacy.render(doc, style='ent', jupyter=True)

### Named Entity Recognition with NLTK

```
nltk.ne_chunk()
```

https://www.nltk.org/book/ch07.html

In [ ]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk.tokenize import sent_tokenize, word_tokenize

sentences = sent_tokenize(text3)
sentences = [word_tokenize(sentence) for sentence in sentences]

# Input to ne_chunk needs to be a part-of-speech tagged word
sentences_pos_tagged = [nltk.pos_tag(word) for word in sentences]

[nltk.ne_chunk(word_pos) for word_pos in sentences_pos_tagged]

## Stop words

Stop words are high-frequency words that don't contribute much lexical content:

- the
- a
- to

NLP libraries usually include a corpus of stop words.

Stop word lists:
- http://www.nltk.org/book/ch02.html#stopwords_index_term
- https://www.semantikoz.com/blog/free-stop-word-lists-in-23-languages/

### Stop words with spaCy

`spacy.lang.en.stop_words`

`token.is_stop`

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

STOP_WORDS

In [ ]:
# Deutsch
from spacy.lang.de.stop_words import STOP_WORDS

STOP_WORDS

In [ ]:
doc = nlp(text3)

for token in doc:
    print(token.text, token.is_stop)

In [ ]:
# Adding stop words
from spacy.lang.en.stop_words import STOP_WORDS

STOP_WORDS.add('MLDDS')

doc = nlp(u"Sorry I'm not free tonite, I have MLDDS (lowercase: mldds).")

for token in doc:
    print(token.text, token.is_stop)

### Stop words with NLTK

```
nltk.corpus.stopwords
```

In [ ]:
# Download corpus
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords.words('english')

In [ ]:
stopwords.words('german')

In [ ]:
tokens = nltk.word_tokenize(text3)

stops = set(stopwords.words('english'))

for token in tokens:
    print(token, token in stops)

In [ ]:
# Adding stop words
stops = stopwords.words('english')
stops.append("MLDDS")
stops = set(stops)

tokens = nltk.word_tokenize(u"Sorry I'm not free tonite, I have MLDDS (lowercase: mldds).")

for token in tokens:
    print(token, token in stops)

# Frequency Analysis

Answers two questions:

1. How often does a word appear in a document?

2. How important is a word in a document?

Measure: Term Frequency - Inverse Document Frequency (TF-IDF)

## Term Frequency

Most common formula:

$$\frac{f_{t, d}}{\sum_{t' \in d} \, f_{t',d}}$$

$f_{t, d}$: count of term $t$ in document $d$

https://en.wikipedia.org/wiki/Tf%E2%80%93idf

## Inverse Document Frequency

Most common formula:

$$log\frac{N}{\mid\{d \in D : t \in d \}\mid}$$

$N$: number of documents

$\mid\{d \in D : t \in d \}\mid$: number of documents containing term $t$

## TD-IDF

$$tfidf(t, d, D) = tf(t, d) * idf(t, D)$$

|term|tf|idf|tf-idf|
|--|--|--|--|--|
|to|large|very small|closer to 0|
|coffee|small|large|not closer to 0|

## Computing TF-IDF

#### Scikit-learn:

```
sklearn.feature_extraction.text.CountVectorizer

sklearn.feature_extraction.text.TfidfVectorizer
```
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


#### NLTK
Supports tf-idf, but less popular
```
nltk.text.TextCollection
```

http://www.nltk.org/api/nltk.html#nltk.text.TextCollection

In [ ]:
text5 = u"This is a test.\n" \
    u"The quick brown fox jumps over the lazy dog.\n" \
    u"The early bird gets the worm.\n"

#### Computing Word Counts

In [ ]:
# http://scikit-learn.org/stable/modules/feature_extraction.html
from sklearn.feature_extraction.text import CountVectorizer

nlp = spacy.load('en_core_web_sm')
doc = nlp(text5)
sentences = [sent.text for sent in doc.sents]

# Count word occurrences
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentences)

# convert sparse matrix to dense matrix
X_dense = X.todense()

X_dense

In [ ]:
vectorizer.get_feature_names()

In [ ]:
# display as a dataframe
import pandas as pd

df_wc = pd.DataFrame(X_dense, columns=vectorizer.get_feature_names())
df_wc

#### Computing TF-IDF

In [ ]:
# http://scikit-learn.org/stable/modules/feature_extraction.html
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer is a combination of
#   CountVectorizer + TfidfTransformer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

# convert sparse matrix to dense matrix
X_dense = X.todense()

print(X_dense.shape)
print(vectorizer.get_feature_names())
X_dense

In [ ]:
# for each sentence, get the highest tf-idf
import numpy as np

terms = vectorizer.get_feature_names()
tfidf_arr = np.array(X_dense)

for i in np.arange(len(sentences)):
    print(sentences[i])
    sorted_idx = np.argsort(tfidf_arr[i])[::-1]
    [print(terms[j], tfidf_arr[i][j]) for j in sorted_idx]
    print()

## Exercise

1. Get 3-5 of your own sample sentences
2. Compute the TF-IDF
3. Compute the TF-IDF with stop_words filtered out:

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

```
from spacy.lang.en.stop_words import STOP_WORDS

vectorizer = TfidfVectorizer(stop_words=STOP_WORDS)

...

```

## N-grams

TF-IDF can be applied to N-grams (N words at a time), to try to capture some context information.

```
CountVectorizer(ngram_range=(minN, maxN)), ..)

TfidfVectorizer(ngram_range=(minN, maxN)), ..)
```

In [ ]:
text5 = u"This is a test.\n" \
    u"The quick brown fox jumps over the lazy dog.\n" \
    u"The early bird gets the worm.\n"

In [213]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

nlp = spacy.load('en_core_web_sm')
doc = nlp(text5)
sentences = [sent.text for sent in doc.sents]

# Count word occurrences using 1 and 2-grams
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(sentences)

# convert sparse matrix to dense matrix
X_dense = X.todense()
print(X_dense.shape)

pd.DataFrame(X_dense, columns=vectorizer.get_feature_names())

(3, 30)


,bird,bird gets,brown,brown fox,dog,early,early bird,fox,fox jumps,gets,...,quick brown,test,the,the early,the lazy,the quick,the worm,this,this is,worm
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,0,0,1,1,1,0,0,1,1,0,...,1,0,2,0,1,1,0,0,0,0
2,1,1,0,0,0,1,1,0,0,1,...,0,0,2,1,0,0,1,0,0,1


## Exercise: TF-IDF with Trigrams

- Compute the TF-IDF for trigrams (1 to 3-grams), using your sample text.
- Try with and without stop words included

In [214]:
# Your code here

from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer is a combination of
#   CountVectorizer + TfidfTransformer
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(sentences)

# convert sparse matrix to dense matrix
X_dense = X.todense()

print(X_dense.shape)
print(vectorizer.get_feature_names())

terms = vectorizer.get_feature_names()
tfidf_arr = np.array(X_dense)

for i in np.arange(len(sentences)):
    print(sentences[i])
    sorted_idx = np.argsort(tfidf_arr[i])[::-1]
    [print(terms[j], tfidf_arr[i][j]) for j in sorted_idx]
    print()

(3, 42)
['bird', 'bird gets', 'bird gets the', 'brown', 'brown fox', 'brown fox jumps', 'dog', 'early', 'early bird', 'early bird gets', 'fox', 'fox jumps', 'fox jumps over', 'gets', 'gets the', 'gets the worm', 'is', 'is test', 'jumps', 'jumps over', 'jumps over the', 'lazy', 'lazy dog', 'over', 'over the', 'over the lazy', 'quick', 'quick brown', 'quick brown fox', 'test', 'the', 'the early', 'the early bird', 'the lazy', 'the lazy dog', 'the quick', 'the quick brown', 'the worm', 'this', 'this is', 'this is test', 'worm']
This is a test.

is test 0.4082482904638631
this is 0.4082482904638631
this 0.4082482904638631
this is test 0.4082482904638631
is 0.4082482904638631
test 0.4082482904638631
early bird gets 0.0
gets the worm 0.0
gets the 0.0
gets 0.0
fox jumps over 0.0
fox jumps 0.0
fox 0.0
worm 0.0
jumps 0.0
early 0.0
dog 0.0
brown fox jumps 0.0
brown fox 0.0
brown 0.0
bird gets the 0.0
bird gets 0.0
early bird 0.0
jumps over the 0.0
jumps over 0.0
lazy 0.0
the worm 0.0
the quick b

# Workshop: Document Summarization